In [63]:

scores = {"a": 1, "c": 3, "b": 3, "e": 1, "d": 2, "g": 2,
"f" : 4, "i": 1, "h": 4, "k": 5, "j": 8, "m": 3,
"l" : 1, "o": 1, "n": 1, "q": 10, "p": 3, "s": 1,
"r" : 1, "u": 1, "t": 1, "w": 4, "v": 4, "y": 4,
"x" : 8, "z": 10}

#Trying stuff out...
print(scores)
print(scores["j"])
boardlist = list('JACQUELINE')
boardlist.remove('E')
print(boardlist)

%pwd


{'s': 1, 'r': 1, 'q': 10, 'p': 3, 'w': 4, 'v': 4, 'u': 1, 't': 1, 'z': 10, 'y': 4, 'x': 8, 'c': 3, 'b': 3, 'a': 1, 'g': 2, 'f': 4, 'e': 1, 'd': 2, 'k': 5, 'j': 8, 'i': 1, 'h': 4, 'o': 1, 'n': 1, 'm': 3, 'l': 1}
8
['J', 'A', 'C', 'Q', 'U', 'L', 'I', 'N', 'E']


'C:\\Users\\Jay\\MIDSPython'

In [276]:
%%writefile scrabble.py
import sys


def findwords(rack, *args):
    """
    rack = string with all letters in a Scrabble rack
    *args = any number of letter/position pairs.  For example,
    "python scrabble.py ZAEFIEE F1 Z3"
    meaning A in 4th position and X in the 10th position.
    Positions are NOT 0-based, because they are meant to be human readable.
    """

    # Index constants for readability
    WORDPOS = 1
    POINTSPOS = 0
    LETTERPOS = 0
    NUMBERPOS = 1

    # Check the args coming in to see if the rack parameter is a string as
    # expected and that the arguments are formatted as (letter)(position)
    try:
        # if type(rack) != type("string"):
        if rack.isinstance(string):
            print("Expecting a string of letters from your scrabble rack to \
                   be the first command line argument")
            return
        else:
            # Addl arguments must have a letter in the first position and
            # an integer after that.
            if args:
                for arg in args:
                    if arg[0].upper() not in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' or \
                      not isinstance(int(arg[1:]), int):
                        print("Letter/Position arguments are expected to be \
                               in (letter)(position) format, i.e. 'A10'")
                        return
    except:
        print("Letter/Position arguments are expected to be in \
               (letter)(position) format, i.e. 'A10'")
        return

    rack = rack.upper()

    words = []
    # will be temporary list of words that can be created (before being
    # sorted and filtered by forced letter positions)

    check = ""
    # word from text file (minus the line feed) to be
    # checked to see if it works

    missing = set()
    # this will be a set of the letters missing from the
    # provided rack (to quickly rule out potential words)

    scores = {"a": 1, "c": 3, "b": 3, "e": 1, "d": 2, "g": 2,
              "f": 4, "i": 1, "h": 4, "k": 5, "j": 8, "m": 3,
              "l": 1, "o": 1, "n": 1, "q": 10, "p": 3, "s": 1,
              "r": 1, "u": 1, "t": 1, "w": 4, "v": 4, "y": 4,
              "x": 8, "z": 10}

    try:
        f = open("sowpods.txt", mode='r')
    except:
        print("Please put the sowpods.txt file in the same directory as \
               this program.")

    if "?" in rack or "*" in rack:
        # make all wildcards * for consistency
        rack = rack.replace("?", "*")
    else:
        missing = set([let for let in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' if
                       let not in rack])

    for row in f:
        check = row.replace("\n", "")    # get rid of the line feeds
        # only look at words short enough to be made and don't contain
        # letters that are unavailable
        if len(check) <= len(rack) and missing - set(check) == missing:

            # temporary rack that we can remove letters from
            racklist = list(rack)

            # will track the letters made with wildcards,
            # since they're not worth any points
            wildcards = []

            # assume that the candidate word works until it doesn't
            nope = False

            # go through the candidate word letter by letter
            for let in check:
                if let in racklist:
                    # spend a tile if it's available
                    racklist.remove(let)
                elif "*" in racklist:
                    # use a wildcard only if necessary
                    racklist.remove("*")
                    wildcards.append(let)
                else:
                    # candidate doesn't work, not enough tiles
                    nope = True
                    break

            if not nope:
                # candidate worked, so calculated the score and display
                # which letters were wildcards
                for w in wildcards:
                    check = check.replace(w, "[" + w.lower() + "]", 1)

                score = 0
                for let in row:
                    if let.lower() in 'abcdefghijklmnopqrstuvwxyz':
                        if let not in wildcards:
                            # Only add points if it wasn't a wildcard
                            score += scores[let.lower()]
                        else:
                            wildcards.remove(let)

                # add the word and the score total to the words list
                words.append([score, check])

    # sort by score and see if they match any filters before printing
    for answer in sorted(words, reverse=True):

        # Assume it works until it doesn't
        nope = False

        for arg in args:
            # The word is shorter than position requirement
            if len(answer[WORDPOS]) < int(arg[NUMBERPOS:]):
                nope = True
            # Does word meet position requirement?
            elif answer[WORDPOS][int(arg[NUMBERPOS:])-1] !=
            arg[LETTERPOS].upper():
                nope = True

        if not nope:
            # The word works!  print the score and the word
            print("".join(str(answer[POINTSPOS]) + " " +
                  answer[WORDPOS].lower()))

# Code starts here when command line is used

print("----------------------------------------------------------------------")
print("Welcome to Jay's amazing Scrabble word-finder!")
print("Provide the letters available as the first command line argument.  You")
print("can also use * or ? as wildcards and add any number of required")
print("letter/position pairs formatted as (letter)(position)")
print("Example: 'python scrabble.py ZAEFIEE F1 Z3'  Note: 1=first position.")
print("----------------------------------------------------------------------")

wildCardCount = 0  # Sounds like user should be limited to two wildcards

if len(sys.argv) < 2:
    print("Expecting a string with letters from your scrabble rack, with \
           ? or * as wildcards)")
else:
    for arg in sys.argv:
        # strip out weird characters
        for let in arg:
            if let.lower() not in 'abcdefghijklmnopqrstuvwxyz*?':
                arg = arg.replace(let, "")
            elif let in '*?':
                wildCardCount += 1

    if wildCardCount > 2:
        print("Only two wildcards are allowed.")
    else:
        if len(sys.argv) == 2:
            findwords(sys.argv[1])
        else:
            # Unpacks the list of additional parameters
            findwords(sys.argv[1], *sys.argv[2:])



Overwriting scrabble.py


In [258]:
findwords("****")

0 [z][z][z][s]
0 [z][z][z]
0 [z][y][m][e]
0 [z][y][g][a]
0 [z][u][z]
0 [z][u][r][f]
0 [z][u][p][a]
0 [z][u][l][u]
0 [z][o][u][k]
0 [z][o][s]
0 [z][o][r][i]
0 [z][o][o][t]
0 [z][o][o][s]
0 [z][o][o][n]
0 [z][o][o][m]
0 [z][o][o]
0 [z][o][n][k]
0 [z][o][n][e]
0 [z][o][n][a]
0 [z][o][l][s]
0 [z][o][l]
0 [z][o][i][c]
0 [z][o][e][a]
0 [z][o][b][u]
0 [z][o][b][o]
0 [z][o][a]
0 [z][o]
0 [z][i][z][z]
0 [z][i][z]
0 [z][i][t][s]
0 [z][i][t][i]
0 [z][i][t][e]
0 [z][i][t]
0 [z][i][p][s]
0 [z][i][p]
0 [z][i][n][s]
0 [z][i][n][g]
0 [z][i][n][e]
0 [z][i][n][c]
0 [z][i][n]
0 [z][i][m][b]
0 [z][i][l][l]
0 [z][i][l][a]
0 [z][i][g][s]
0 [z][i][g]
0 [z][i][f][f]
0 [z][h][o][s]
0 [z][h][o]
0 [z][e][z][e]
0 [z][e][x]
0 [z][e][t][a]
0 [z][e][s][t]
0 [z][e][r][o]
0 [z][e][r][k]
0 [z][e][p][s]
0 [z][e][p]
0 [z][e][l][s]
0 [z][e][l]
0 [z][e][k][s]
0 [z][e][k]
0 [z][e][i][n]
0 [z][e][e][s]
0 [z][e][e]
0 [z][e][d][s]
0 [z][e][d]
0 [z][e][b][u]
0 [z][e][a][s]
0 [z][e][a][l]
0 [z][e][a]
0 [z][a][x]
0 [z][a][t][i]
0